# Homework 4: Fun with Point Clouds?

## Problem 1: Point Cloud Fusion

### Part 1: Harris Corner Detection

#### Code Imports

In [30]:
# optional: allow Jupyter to "hot reload" the modules we import - after each change, rerun this cell (instead of restarting the kernel!!)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import glob

import numpy as np

from util import ops
from util.corner_detection import HarrisCornerDetector

### Load Data

In [20]:
rgb_paths = sorted(glob.glob("../hw4_data/problem1/rgb*"))
rgb_paths

['../hw4_data/problem1/rgb1.png',
 '../hw4_data/problem1/rgb2.png',
 '../hw4_data/problem1/rgb3.png']

In [21]:
rgb_images = [
    ops.load_image(
        path,
        return_grayscale=False,
        return_array=True
    )
    for path in rgb_paths
]

Dimensions of ../hw4_data/problem1/rgb1.png: 480 x 640 x 3
Dimensions of ../hw4_data/problem1/rgb2.png: 480 x 640 x 3
Dimensions of ../hw4_data/problem1/rgb3.png: 480 x 640 x 3


For convenience, let's map the filename of each RGB image, to the image itself.

And let's also load in the depth map images in a similar fashion:

In [22]:
depthmap_paths = sorted(glob.glob("../hw4_data/problem1/depth*"))
depthmap_paths

['../hw4_data/problem1/depth1.png',
 '../hw4_data/problem1/depth2.png',
 '../hw4_data/problem1/depth3.png']

In [23]:
depthmap_paths = [
    ops.load_image(
        path,
        return_grayscale=True,
        return_array=True
    )
    for path in depthmap_paths
]

Dimensions of ../hw4_data/problem1/depth1.png: 480 x 640
Dimensions of ../hw4_data/problem1/depth2.png: 480 x 640
Dimensions of ../hw4_data/problem1/depth3.png: 480 x 640


#### Determine Feature that Will Be Used to Fuse Point Clouds 

In [27]:
derivative_operator_x = np.array([
    [-1, 0, 1],
    [-1, 0, 1],
    [-1, 0, 1],
])
derivative_operator_y = derivative_operator_x.T

In [28]:
# using the Gaussian filter provided here: https://homepages.inf.ed.ac.uk/rbf/HIPR2/gsmooth.htm
gaussian_window = np.array([
    [1, 4, 7, 4, 1],
    [4, 16, 26, 16, 4],
    [7, 26, 41, 26, 7],
    [4, 16, 26, 16, 4],
    [1, 4, 7, 4, 1],
])

gaussian_window = gaussian_window * (1 / 273)

In [32]:
rgb_path_to_image_and_corners = dict()

detector = HarrisCornerDetector()
top_many_features = 100

for rgb_path, rgb_image in zip(rgb_paths, rgb_images):

    corner_response = detector.detect_features(
        rgb_image,
        use_non_max_suppression=True,
        derivative_operator_x=derivative_operator_x,
        derivative_operator_y=derivative_operator_y,
        gaussian_window=gaussian_window,
    )
    # pick top features
    top_k_points = detector.pick_top_features(
        corner_response,
        top_many_features
    )

    rgb_path_to_image_and_corners[rgb_path] = (rgb_image, top_k_points)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)